# Transfer Learning Models

## Libraries to Import

In [18]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from PIL import Image
import keras
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization

import os
import gc
from timeit import default_timer as timer

## Define Image Directories

In [4]:
train_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/training/'
test_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/testing/'
val_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/validation/'

Create flow_from_directory objects for Keras model. Using a downsized dataset with class balance for local model running.

In [5]:
train_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(train_dir,
                                                                  target_size = (256,256), 
                                                                  color_mode='rgb',
                                                                  class_mode='binary')
val_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(val_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')
test_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(test_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')

Found 1346 images belonging to 2 classes.
Found 593 images belonging to 2 classes.
Found 947 images belonging to 2 classes.


## Xception

In [6]:
from keras.applications.vgg16 import VGG16

In [19]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(450, 450, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

83689472/83683744 [==============================] - 3s 0us/step


In [22]:
inputs = keras.Input(shape=(450, 450, 3))
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation ='sigmoid')(x)
model = keras.Model(inputs, outputs)

In [23]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Recall'])
start = timer()
model.fit(train_g, 
            epochs=5, 
            validation_data=val_g, 
            workers = 7)
end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 140s 3s/step - loss: 0.6371 - accuracy: 0.6612 - recall: 0.6657 - val_loss: 0.5866 - val_accuracy: 0.7285 - val_recall: 0.7381
Epoch 2/5
43/43 [==============================] - 157s 4s/step - loss: 0.5689 - accuracy: 0.7348 - recall: 0.7504 - val_loss: 0.5515 - val_accuracy: 0.7369 - val_recall: 0.7517
Epoch 3/5
43/43 [==============================] - 149s 3s/step - loss: 0.5432 - accuracy: 0.7422 - recall: 0.7221 - val_loss: 0.5315 - val_accuracy: 0.7521 - val_recall: 0.7789
Epoch 4/5
43/43 [==============================] - 147s 3s/step - loss: 0.5259 - accuracy: 0.7600 - recall: 0.7340 - val_loss: 0.5498 - val_accuracy: 0.7167 - val_recall: 0.8946
Epoch 5/5
43/43 [==============================] - 145s 3s/step - loss: 0.5078 - accuracy: 0.7615 - recall: 0.7563 - val_loss: 0.5155 - val_accuracy: 0.7589 - val_recall: 0.7891
Total Time Elapsed:  12  minutes  34  seconds


In [24]:
scores = model.evaluate(test_g, verbose=1)

30/30 [==============================] - 65s 2s/step - loss: 0.9548 - accuracy: 0.4340 - recall: 0.2685
